In [1]:
%matplotlib inline 

from weibo.get_weibo import get_weibo
from weibo.baidu_nltk import get_sentiment_simple, get_sentiment_positive_proba, tokenize,tokenize_df

import pandas as pd
import numpy as np
import utilities.misc as um
import utilities.constants as uc
import utilities.display as ud

import time

from wordcloud import WordCloud
from scipy.misc import imread


import matplotlib.pyplot as plt

import feather

In [2]:
cookie='SINAGLOBAL=1939137473236.7695.1359786817801; wvr=6; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5GrRHKMLMbdJMQDxRPi1695JpX5KMhUgL.FoM01h-N1hBce052dJLoI0jLxK-LBKBLB-2LxK.L1KnLBoeLxKnL1heLB.BLxKqL1h.L12zLxKML1KBLBo-LxKMLB-BL1KzEShM01Btt; ALF=1612831058; SSOLoginState=1581295060; SCF=AuNRrR3OOAeW3qjtI0IQx4C0_QxU1OBOXeuhJKBl0eRSNlAlwOT6GiC2zn7dToLUcdfEI3X309SA_l3JQfjfr-8.; SUB=_2A25zRNWEDeThGeFN41cW-CrKyDyIHXVQMEBMrDV8PUNbmtAfLXn2kW9NQ9OAjkhevz9hAwDzhQv2o3AJnCDaDuFF; SUHB=0tK6O9Vw5tLFhT; _s_tentry=login.sina.com.cn; Apache=3624480979299.9688.1581295297607; ULV=1581295297645:13:7:1:3624480979299.9688.1581295297607:1581155001927; webim_unReadCount=%7B%22time%22%3A1581295297677%2C%22dm_pub_total%22%3A1%2C%22chat_group_client%22%3A0%2C%22allcountNum%22%3A31%2C%22msgbox%22%3A0%7D; UOR=cn.wsj.com,widget.weibo.com,www.google.com; secsys_id=c0f758526f290668b625d1e4f43875ce'

In [3]:
path="D:\\data\\weibo\\"
dump_path="D:\\data\\weibo\\dump\\%s.feather"

In [4]:
# manually control everything here
topics=['冠状病毒'] #'武汉'
file_name_dict={'冠状病毒':'guanzhuangbingdu','口罩':'kouzhao','医生':'yisheng','野味':'yewei','湖北':'hubei','武汉':'wuhan','疫情':'yiqing'}
start=pd.datetime(2020,1,19)
end=um.yesterday_date()
freq='D'
dates=pd.date_range(start,end,freq=freq)

In [11]:
collector=[]
for topic in topics:
    for date in dates:
        weibo_i=get_weibo(topic,date,cookie)
        if weibo_i is not False:
            file_name='%s_%s' % (file_name_dict[topic],date.strftime('%Y-%m-%d'))
            feather.write_dataframe(weibo_i,dump_path % (file_name))
            print ('@@@ %s finished' % (file_name))

        

Start working on topic 冠状病毒 on 2020-01-19
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Finish page 8
Finish page 9
Finish page 10
Finish page 11
Finish page 12
Finish page 13
Finish page 14
Finish page 15
Finish page 16
Page 16 is the end page
@@@ guanzhuangbingdu_2020-01-19 finished
Start working on topic 冠状病毒 on 2020-01-20
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Finish page 8
Finish page 9
Finish page 10
Finish page 11
Finish page 12
Finish page 13
Finish page 14
Finish page 15
Finish page 16
Finish page 17
Finish page 18
Finish page 19
Finish page 20
Finish page 21
Finish page 22
Page 22 is the end page
@@@ guanzhuangbingdu_2020-01-20 finished
Start working on topic 冠状病毒 on 2020-01-21
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Finish page 8
Finish page 9
Finish page 10
Finish page 11
Finish page 12
Finish page 13
Finish

Finish page 10
Finish page 11
Finish page 12
Finish page 13
Finish page 14
Finish page 15
Finish page 16
Page 16 is the end page
@@@ guanzhuangbingdu_2020-02-07 finished


In [22]:
# re-collect and dump the data
all_files=um.iterate_csv(dump_path.replace('%s.feather',''),iterate_others=[True,'.feather'])
collector=[]
for file in all_files:
    df_i=feather.read_dataframe(dump_path % (file))
    collector.append(df_i)
data=pd.concat(collector,axis=0)


In [23]:
# save the data
feather.write_dataframe(data,path+'data.feather')

In [5]:
# get the word tokenization
data=feather.read_dataframe(path+'data.feather')

collector=[]
dates=data.groupby(['date','topic']).last().index.levels[0]
topics=data.groupby(['date','topic']).last().index.levels[1]
for date in dates:
    for topic in topics:
        words_i=tokenize_df(data[(data['topic']==topic) & (data['date']==date)],'content')
        if words_i is not False:
            words_i['date']=date
            words_i['topic']=topic
            collector.append(words_i)
            print ('finish %s on %s' % (topic,date))

words_all=pd.concat(collector,axis=0)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.898 seconds.
Prefix dict has been built successfully.


finish 冠状病毒 on 2019-12-01 00:00:00
finish 武汉 on 2019-12-01 00:00:00
finish 武汉 on 2019-12-02 00:00:00
finish 武汉 on 2019-12-03 00:00:00
finish 武汉 on 2019-12-04 00:00:00
finish 武汉 on 2019-12-05 00:00:00
finish 武汉 on 2019-12-06 00:00:00
finish 武汉 on 2019-12-07 00:00:00
finish 武汉 on 2019-12-08 00:00:00
finish 武汉 on 2019-12-09 00:00:00
finish 冠状病毒 on 2019-12-10 00:00:00
finish 武汉 on 2019-12-10 00:00:00
finish 冠状病毒 on 2019-12-11 00:00:00
finish 武汉 on 2019-12-11 00:00:00
finish 冠状病毒 on 2019-12-12 00:00:00
finish 武汉 on 2019-12-12 00:00:00
finish 武汉 on 2019-12-13 00:00:00
finish 武汉 on 2019-12-14 00:00:00
finish 武汉 on 2019-12-15 00:00:00
finish 武汉 on 2019-12-16 00:00:00
finish 冠状病毒 on 2019-12-17 00:00:00
finish 武汉 on 2019-12-17 00:00:00
finish 武汉 on 2019-12-18 00:00:00
finish 武汉 on 2019-12-19 00:00:00
finish 武汉 on 2019-12-20 00:00:00
finish 武汉 on 2019-12-21 00:00:00
finish 武汉 on 2019-12-22 00:00:00
finish 武汉 on 2019-12-23 00:00:00
finish 武汉 on 2019-12-24 00:00:00
finish 武汉 on 2019-12-25 00:00:00


In [6]:
feather.write_dataframe(words_all,path+'words.feather')

In [32]:
# reload words, and pick the top 5pct and dump xlsx for google translate through web
pct=0.95

words=feather.read_dataframe(path+'words.feather')
count=words.groupby(['topic','date','item']).count().reset_index()
count['rank']=count.groupby(['topic','date'])['prop'].rank(pct=True)
count=count[count['rank']>pct]

count.to_excel(path+'words_translation.xlsx')

In [35]:
# reloads words,add sentiment and dump
data=feather.read_dataframe(path+'data.feather')
data['sentiment']=data['content'].map(lambda x: get_sentiment_simple(x))

In [36]:
feather.write_dataframe(data,path+'data_with_sentiment.feather')